In [1]:
!pip install requests BeautifulSoup4 fire

  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=dc958b0f9ad5056ce421c51e5dfafd56277765fb61b61647d11113ec127dfc47
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\1f\10\06\2a990ee4d73a8479fe2922445e8a876d38cfbfed052284c6a1
Successfully built fire


In [2]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd
import os, sys

import fire

In [5]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content  #.encode(BeautifulSoup.original_encoding)
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)
    


def get_elements(url, tag='',search={}, fname=None):
    """
    Downloads a page specified by the url parameter
    and returns a list of strings, one per tag element
    """
    
    if isinstance(url,str):
        response = simple_get(url)
    else:
        #if already it is a loaded html page
        response = url

    if response is not None:
        html = BeautifulSoup(response, 'html.parser')
        
        res = []
        if tag:    
            for li in html.select(tag):
                for name in li.text.split('\n'):
                    if len(name) > 0:
                        res.append(name.strip())
                       
                
        if search:
            soup = html            
            
            
            r = ''
            if 'find' in search.keys():
                print('findaing',search['find'])
                soup = soup.find(**search['find'])
                r = soup

                
            if 'find_all' in search.keys():
                print('finding all of',search['find_all'])
                r = soup.find_all(**search['find_all'])
   
            if r:
                for x in list(r):
                    if len(x) > 0:
                        res.extend(x)
            
        return res

    # Raise an exception if we failed to get any data from the url
    raise Exception('Error retrieving contents at {}'.format(url))    
    
    
if get_ipython().__class__.__name__ == '__main__':
    fire(get_tag_elements)

In [97]:
import pandas as pd
end_date = get_elements('https://en.wikipedia.org/wiki/COVID-19_lockdowns',tag='#mw-content-text > div.mw-parser-output > div:nth-child(50) > table > tbody > tr:nth-child(95)')
end_date1 = get_elements('https://en.wikipedia.org/wiki/COVID-19_lockdowns',tag='#mw-content-text > div.mw-parser-output > div:nth-child(50) > table > tbody > tr:nth-child(94)')
data = pd.DataFrame(end_date)
data1 = pd.DataFrame(end_date1)
all_data = pd.concat([data,data1], axis=1)
all_data

,0,0
0,Paris,France
1,2021-03-19[252],Nationwide
2,2021-04-18,2020-03-17[247]
3,30,2020-05-11[248]
4,131,55
5,Region,2020-10-30[249]
6,NaN,2020-12-15[250]
7,NaN,46
8,NaN,2021-04-04[251]
9,NaN,2021-05-03[251]


In [16]:
df = pd.DataFrame(end_date)
df.head(15)

TypeError: __init__() got an unexpected keyword argument 'skiprows'

In [15]:
type(end_date)

list